In [3]:
#PreProcessing
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
import joblib
import warnings
warnings.filterwarnings('ignore')
#Feature Engineering
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from xgboost import XGBClassifier        # recommended for feature importance
from catboost import CatBoostClassifier  # optional alternative, handles categorical features natively
#visualization
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap

In [2]:
df = pd.read_csv("D:/Machine Learning Projects/5. Behavioral Segmentation of Retail Customers using Clustering/0_OR_clean.csv")

# Pre-Processing

In [3]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01/12/2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01/12/2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01/12/2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01/12/2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01/12/2010 08:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,09/12/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,09/12/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,09/12/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,09/12/2011 12:50,4.15,12680.0,France


#### Missing Values

In [4]:
missing_values = df.isna().sum()
print(missing_values)

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


In [5]:
df_clean = df.dropna(subset=['CustomerID'])

In [6]:
missing_values = df_clean.isna().sum()
print(missing_values)

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64


#### Duplicate Records

In [7]:
df_clean.duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
541904    False
541905    False
541906    False
541907    False
541908    False
Length: 406829, dtype: bool

In [8]:
duplicate_count = df_clean.duplicated().sum()
print("Number of duplicate rows:", duplicate_count)

Number of duplicate rows: 5225


In [9]:
duplicates = df_clean[df_clean.duplicated()]
duplicates.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
517,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,01/12/2010 11:45,1.25,17908.0,United Kingdom
527,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,01/12/2010 11:45,2.10,17908.0,United Kingdom
537,536409,22900,SET 2 TEA TOWELS I LOVE LONDON,1,01/12/2010 11:45,2.95,17908.0,United Kingdom
539,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,01/12/2010 11:45,4.95,17908.0,United Kingdom
555,536412,22327,ROUND SNACK BOXES SET OF 4 SKULLS,1,01/12/2010 11:49,2.95,17920.0,United Kingdom


In [10]:
df_clean = df_clean.drop_duplicates()

In [11]:
duplicate_count = df_clean.duplicated().sum()
print("Number of duplicate rows:", duplicate_count)

Number of duplicate rows: 0


#### Noise/Outliers

In [12]:
for col in df.columns:
    print(f"{col}: {df[col].unique()}")

InvoiceNo: ['536365' '536366' '536367' ... '581585' '581586' '581587']
StockCode: ['85123A' '71053' '84406B' ... '90214U' '47591b' '23843']
Description: ['WHITE HANGING HEART T-LIGHT HOLDER' 'WHITE METAL LANTERN'
 'CREAM CUPID HEARTS COAT HANGER' ... 'lost'
 'CREAM HANGING HEART T-LIGHT HOLDER' 'PAPER CRAFT , LITTLE BIRDIE']
Quantity: [     6      8      2     32      3      4     24     12     48     18
     20     36     80     64     10    120     96     23      5      1
     -1     50     40    100    192    432    144    288    -12    -24
     16      9    128     25     30     28      7     56     72    200
    600    480     -6     14     -2     11     33     13     -4     -5
     -7     -3     70    252     60    216    384    -10     27     15
     22     19     17     21     34     47    108     52  -9360    -38
     75    270     42    240     90    320   1824    204     69    -36
   -192   -144    160   2880   1400     39    -48    -50     26   1440
     31     82     78   

In [13]:
print(df_clean['Quantity'].describe())

count    401604.000000
mean         12.183273
std         250.283037
min      -80995.000000
25%           2.000000
50%           5.000000
75%          12.000000
max       80995.000000
Name: Quantity, dtype: float64


In [14]:
negative_qty = df_clean[df_clean['Quantity'] < 0]
print("Negative Quantity rows:", negative_qty.shape[0])

Negative Quantity rows: 8872


In [15]:
large_qty = df_clean[df_clean['Quantity'] > df_clean['Quantity'].quantile(0.99)]
print("Top 1% largest quantities:", large_qty.shape[0])

Top 1% largest quantities: 3894


In [28]:
def classify_transaction(qty):
    if qty > 0:
        return 'Sale'
    elif qty < 0:
        return 'Return'
    else:
        return 'Not Sale'

df_clean['TransactionType'] = df_clean['Quantity'].apply(classify_transaction)
print(df_clean['TransactionType'].value_counts())

TransactionType
Sale      392692
Return      8872
Name: count, dtype: int64


In [17]:
print(df_clean['UnitPrice'].describe())

count    401604.000000
mean          3.474064
std          69.764035
min           0.000000
25%           1.250000
50%           1.950000
75%           3.750000
max       38970.000000
Name: UnitPrice, dtype: float64


In [18]:
invalid_price = df_clean[df_clean['UnitPrice'] <= 0]
print("Zero or negative UnitPrice rows:", invalid_price.shape[0])

Zero or negative UnitPrice rows: 40


In [20]:
df_clean = df_clean[df_clean['UnitPrice'] > 0]

In [19]:
high_price = df_clean[df_clean['UnitPrice'] > df_clean['UnitPrice'].quantile(0.99)]
print("Top 1% highest prices:", high_price.shape[0])

Top 1% highest prices: 3857


In [22]:
canceled_invoices = df_clean[df_clean['InvoiceNo'].astype(str).str.startswith('C')]

print("Number of canceled invoices:", canceled_invoices['InvoiceNo'].nunique())
print("Number of canceled rows:", len(canceled_invoices))

Number of canceled invoices: 3654
Number of canceled rows: 8872


In [23]:
non_numeric_codes = df_clean[~df_clean['StockCode'].str.isnumeric()]['StockCode'].unique()

print("Unique non-numeric StockCodes:")
print(non_numeric_codes)
print("Count:", len(non_numeric_codes))

Unique non-numeric StockCodes:
['85123A' '84406B' '84029G' '84029E' 'POST' '82494L' '85099C' '84997B'
 '84997C' '84519A' '85183B' '85071B' '37444A' '37444C' '84971S' '15056BL'
 '15056N' 'D' '35004C' '85049A' '85099B' '35004G' '85014B' '85014A'
 '84970S' '84030E' '35004B' '85049E' '17091A' '84509A' '84510A' '84709B'
 '84625C' '84625A' '47570B' '85049C' '85049D' '85049G' '84970L' '90199C'
 '90129F' '90210B' '72802C' '85169B' '85099F' '85184C' '35591T' '84032B'
 '85049H' '72800E' '84849B' '90200B' '90059B' '90185C' '90059E' '90059C'
 '90200C' '90200D' '90200A' '16258A' '85231B' '85231G' '48173C' '47563A'
 '84558A' '46000M' '71406C' '84985A' '84596E' '84997D' '47599A' '47599B'
 '85035B' '84968C' '72800B' '84563A' '47504H' '17164B' '15044B' '84569B'
 '85114B' '85114C' '85199L' '85199S' '85019A' '85019C' '85071A' '85071C'
 '85135B' '85136A' '85136C' 'C2' '79144B' '46000R' '46000S' '84508A'
 '85232B' '79066K' '84884A' '51014C' '51014L' '51014A' '79302M' '84509B'
 '84870C' '84870B' 'M' '85032D

In [31]:
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'], dayfirst=True, errors='coerce')
df_clean['InvoiceYear'] = df_clean['InvoiceDate'].dt.year
df_clean['InvoiceMonth'] = df_clean['InvoiceDate'].dt.month
df_clean['InvoiceDay'] = df_clean['InvoiceDate'].dt.day
df_clean['InvoiceHour'] = df_clean['InvoiceDate'].dt.hour
df_clean['InvoiceWeekday'] = df_clean['InvoiceDate'].dt.weekday  

In [32]:
invalid_dates = df_clean[df_clean['InvoiceDate'].isna()]
print(f"Number of invalid dates: {len(invalid_dates)}")
print(invalid_dates.head())

Number of invalid dates: 0
Empty DataFrame
Columns: [InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country, InvoiceYear, InvoiceMonth, InvoiceDay, InvoiceHour, InvoiceWeekday, TransactionType]
Index: []


In [33]:
df_clean.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceYear,InvoiceMonth,InvoiceDay,InvoiceHour,InvoiceWeekday,TransactionType
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010,12,1,8,2,Sale
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010,12,1,8,2,Sale
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010,12,1,8,2,Sale
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010,12,1,8,2,Sale
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010,12,1,8,2,Sale


In [34]:
df_clean.to_csv('D:/Machine Learning Projects/5. Behavioral Segmentation of Retail Customers using Clustering/df_clean.csv', index=False)

### Encoding

In [4]:
df_clean = pd.read_csv("D:/Machine Learning Projects/5. Behavioral Segmentation of Retail Customers using Clustering/df_clean.csv")

In [5]:
df_clean = df_clean.drop(columns=['InvoiceDate'])

In [6]:
cat_cols = ['InvoiceNo', 'StockCode', 'Description', 'Country', 'TransactionType']
for col in cat_cols:
    le = LabelEncoder()
    df_clean[col] = le.fit_transform(df_clean[col])
    
    joblib.dump(le, f"D:/Machine Learning Projects/5. Behavioral Segmentation of Retail Customers using Clustering/le_{col}.pkl")

In [8]:
df_clean.head(20)

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,InvoiceYear,InvoiceMonth,InvoiceDay,InvoiceHour,InvoiceWeekday,TransactionType
0,0,3249,3716,6,2.55,17850.0,35,2010,12,1,8,2,1
1,0,2649,3724,6,3.39,17850.0,35,2010,12,1,8,2,1
2,0,2855,861,8,2.75,17850.0,35,2010,12,1,8,2,1
3,0,2803,1813,6,3.39,17850.0,35,2010,12,1,8,2,1
4,0,2802,2776,6,3.39,17850.0,35,2010,12,1,8,2,1
5,0,1605,2977,2,7.65,17850.0,35,2010,12,1,8,2,1
6,0,755,1444,6,4.25,17850.0,35,2010,12,1,8,2,1
7,1,1490,1562,6,1.85,17850.0,35,2010,12,1,8,2,1
8,1,1489,1559,6,1.85,17850.0,35,2010,12,1,8,2,1
9,2,3069,217,32,1.69,13047.0,35,2010,12,1,8,2,1


In [9]:
df_clean.to_csv('D:/Machine Learning Projects/5. Behavioral Segmentation of Retail Customers using Clustering/df_encoded.csv', index=False)

### Scaling

In [10]:
scaler = MinMaxScaler()
df_clean[:] = scaler.fit_transform(df_clean)

joblib.dump(scaler, "D:/Machine Learning Projects/5. Behavioral Segmentation of Retail Customers using Clustering/scaler.pkl")

['D:/Machine Learning Projects/5. Behavioral Segmentation of Retail Customers using Clustering/scaler.pkl']

In [11]:
df_clean.head()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,InvoiceYear,InvoiceMonth,InvoiceDay,InvoiceHour,InvoiceWeekday,TransactionType
0,0.0,0.882161,0.954044,0.500037,0.000065,0.926443,0.972222,0,1.0,0.0,0.142857,0.333333,1
1,0.0,0.719251,0.956098,0.500037,0.000087,0.926443,0.972222,0,1.0,0.0,0.142857,0.333333,1
2,0.0,0.775183,0.221053,0.500049,0.000071,0.926443,0.972222,0,1.0,0.0,0.142857,0.333333,1
3,0.0,0.761064,0.465469,0.500037,0.000087,0.926443,0.972222,0,1.0,0.0,0.142857,0.333333,1
4,0.0,0.760793,0.712709,0.500037,0.000087,0.926443,0.972222,0,1.0,0.0,0.142857,0.333333,1


In [12]:
df_clean.to_csv('D:/Machine Learning Projects/5. Behavioral Segmentation of Retail Customers using Clustering/df_scaled.csv', index=False)